In [7]:
import numpy as np
import os
import cv2
from tqdm import tqdm
import math
import matplotlib.pyplot as plt
import glob

In [8]:
path = 'test_results_Sony/'
file_path_gt = []
file_path_out = []

for img in os.listdir(path):
    file_path = img
    
    if 'gt' in file_path:
        file_path_gt.append(file_path)
    elif 'out' in file_path:
        file_path_out.append(file_path)
    else:
        continue
        
print(len(file_path_gt))
print(len(file_path_out))

93
93


In [9]:
file_path_gt.sort()
file_path_out.sort()

print(file_path_gt)
print(file_path_out)

['10003_00_100_gt.png', '10003_00_250_gt.png', '10006_00_100_gt.png', '10006_00_250_gt.png', '10011_00_100_gt.png', '10011_00_250_gt.png', '10016_00_100_gt.png', '10016_00_250_gt.png', '10022_00_100_gt.png', '10022_00_250_gt.png', '10030_00_100_gt.png', '10030_00_250_gt.png', '10032_00_100_gt.png', '10032_00_250_gt.png', '10034_00_100_gt.png', '10034_00_250_gt.png', '10035_00_100_gt.png', '10035_00_250_gt.png', '10040_00_100_gt.png', '10040_00_250_gt.png', '10045_00_100_gt.png', '10045_00_250_gt.png', '10054_00_100_gt.png', '10054_00_250_gt.png', '10055_00_100_gt.png', '10055_00_250_gt.png', '10068_00_100_gt.png', '10068_00_250_gt.png', '10069_00_100_gt.png', '10069_00_250_gt.png', '10074_00_300_gt.png', '10077_00_300_gt.png', '10082_00_300_gt.png', '10087_00_300_gt.png', '10093_00_300_gt.png', '10101_00_300_gt.png', '10103_00_300_gt.png', '10105_00_300_gt.png', '10106_00_300_gt.png', '10111_00_300_gt.png', '10116_00_300_gt.png', '10125_00_300_gt.png', '10126_00_300_gt.png', '10139_00_

In [10]:
def calc_psnr(img1, img2):
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    if img1.ndim == 2:
        mse = np.mean((img1 - img2)**2)
    elif img1.ndim == 3:
        mse1 = np.mean((img1[:,:,0] - img2[:,:,0])**2)
        mse2 = np.mean((img1[:,:,1] - img2[:,:,1])**2)
        mse3 = np.mean((img1[:,:,2] - img2[:,:,2])**2)
        mse = (mse1 + mse2 + mse3)/3
    if mse == 0:
        return float('inf')
    psnr = 20 * math.log10(255.0 / math.sqrt(mse))
    return psnr

In [11]:
psnr_fin = 0

for i in tqdm(range(len(file_path_gt))):
    path1 = file_path_gt[i]
    path2 = file_path_out[i]
    
    img1 = cv2.imread(os.path.join(path,path1))
    img2 = cv2.imread(os.path.join(path,path2))
    psnr_fin += calc_psnr(img1, img2)
    
psnr_fin = (psnr_fin)/(len(file_path_gt))
print(psnr_fin)

100%|██████████| 93/93 [02:06<00:00,  1.36s/it]

24.84174882586936


In [12]:
def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()

In [13]:
def calc_ssim(img1, img2):
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

In [14]:
ssim_fin = 0

for i in tqdm(range(len(file_path_gt))):
    path1 = file_path_gt[i]
    path2 = file_path_out[i]
    
    img1 = cv2.imread(os.path.join(path,path1))
    img2 = cv2.imread(os.path.join(path,path2))
    ssim_fin += calc_ssim(img1, img2)
    
ssim_fin = (ssim_fin)/(len(file_path_gt))
print(ssim_fin)

100%|██████████| 93/93 [24:56<00:00, 16.09s/it]

0.6489079481518409
